In [8]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [9]:
spark = (
    SparkSession.builder
    .appName("Curso de PySpark")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .getOrCreate()
)

In [10]:
spark

In [11]:
path = "C:/Users/marlos.barros/Cursos/pyspark_na_pratica/DATASETS/LOGINS.parquet"

In [16]:
df = spark.read.format("parquet").load(path).limit(5)

In [17]:
df.printSchema()

root
 |-- cpf: string (nullable = true)
 |-- email: string (nullable = true)
 |-- senha: string (nullable = true)
 |-- data_de_nascimento: date (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_cadastro: date (nullable = true)
 |-- ipv4: string (nullable = true)
 |-- cor_favorita: string (nullable = true)
 |-- profissao: string (nullable = true)
 |-- telefone: string (nullable = true)



### Criação da Coluna num com dayofmonth
- Função: Cria uma nova coluna chamada **"num"** que extrai o **dia** do **mês** da coluna **"data_de_nascimento"**. Por exemplo, se a **data de nascimento** for** 2025-03-17**, a coluna **"num"** conterá o valor **17**.

In [18]:
df = df.withColumn("num", F.dayofmonth(F.col("data_de_nascimento")))

In [19]:
df.printSchema()

root
 |-- cpf: string (nullable = true)
 |-- email: string (nullable = true)
 |-- senha: string (nullable = true)
 |-- data_de_nascimento: date (nullable = true)
 |-- estado: string (nullable = true)
 |-- data_cadastro: date (nullable = true)
 |-- ipv4: string (nullable = true)
 |-- cor_favorita: string (nullable = true)
 |-- profissao: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- num: integer (nullable = true)



### Agrupamento por **cor_favorita** com várias agregações
- Função: Agrupa o DataFrame pela coluna **"cor_favorita"** e aplica várias funções de agregação na coluna **"num"**:
    - **F.count("*")**: Conta o número **total** de registros em cada grupo de **"cor_favorita"**.
    - **F.sum("num")**: Soma os valores da coluna **"num"** para cada grupo de **"cor_favorita"**.
    - **F.avg("num")**: Calcula a **média** dos valores de **"num"** em cada grupo de **"cor_favorita"**.
    - **F.max("num")**: Retorna o **maior** valor de **"num"** para cada grupo.
    - **F.min("num")**: Retorna o **menor** valor de **"num"** para cada grupo.
    - **F.stddev("num")**: Calcula o **desvio padrão** dos valores de **"num"** para cada grupo.

In [20]:
df.groupBy(F.col("cor_favorita")).agg(F.count("*"), F.sum("num"), F.avg("num"), F.max("num"), F.min("num"), F.stddev("num"))

cor_favorita,count(1),sum(num),avg(num),max(num),min(num),stddev(num)
Roxo,1,18,18.0,18,18,NULL
Ciano,1,17,17.0,17,17,NULL
Azul,1,11,11.0,11,11,NULL
Marrom,1,1,1.0,1,1,NULL
Laranja,1,28,28.0,28,28,NULL


### Agrupamento por **cor_favorita** com dicionário de agregações
- Função: Este comando também agrupa o DataFrame pela coluna **"cor_favorita"**, mas utiliza um **dicionário** para definir as agregações.
    - **"*": "count"**: **Conta** o número **total** de registros para cada grupo de **"cor_favorita"**.
    - **"num": "sum"**: **Soma** os valores da coluna **"num"** para cada grupo de **"cor_favorita"**.

In [21]:
df.groupBy(F.col("cor_favorita")).agg({"*": "count", "num": "sum"})

cor_favorita,sum(num),count(1)
Roxo,18,1
Ciano,17,1
Azul,11,1
Marrom,1,1
Laranja,28,1
